In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('formula1.db')

points_query = '''
    SELECT r.year, ds.driverId, d.forename || ' ' || d.surname AS driver_name, ds.points
    FROM driver_standings ds
    INNER JOIN (
        SELECT r1.year, MAX(r1.date) AS last_race_date
        FROM races r1
        GROUP BY r1.year
    ) last_race ON ds.raceId = (
        SELECT r2.raceId
        FROM races r2
        WHERE r2.year = last_race.year AND r2.date = last_race.last_race_date
        LIMIT 1
    )
    INNER JOIN races r ON ds.raceId = r.raceId AND r.date = last_race.last_race_date
    INNER JOIN drivers d ON ds.driverId = d.driverId;
'''

points_df = pd.read_sql_query(points_query, conn)

points_df = points_df.sort_values(by=['year', 'points'], ascending=[True, False])

points_df['place'] = points_df.groupby('year')['points'].rank(ascending=False, method='min').astype(int)

# Display the DataFrame
print("Points DataFrame with Place:")
display(points_df)

# Save the DataFrame to a CSV file
csv_file_path_reshaped = 'enaa.csv'
points_df.to_csv(csv_file_path_reshaped, index=False)


Points DataFrame with Place:


,year,driverId,driver_name,points,place
73,1950,642,Nino Farina,30.0,1
66,1950,579,Juan Fangio,27.0,2
72,1950,786,Luigi Fagioli,24.0,3
70,1950,627,Louis Rosier,13.0,4
56,1950,647,Alberto Ascari,11.0,5
...,...,...,...,...,...
3133,2022,855,Guanyu Zhou,6.0,18
3136,2022,848,Alexander Albon,4.0,19
3139,2022,849,Nicholas Latifi,2.0,20
3145,2022,856,Nyck de Vries,2.0,20


In [ ]:
def calculate_final_driver_places(conn):

    points_query = '''
        SELECT r.year, ds.driverId, d.forename || ' ' || d.surname AS driver_name, ds.points
        FROM driver_standings ds
        INNER JOIN (
            SELECT r1.year, MAX(r1.date) AS last_race_date
            FROM races r1
            GROUP BY r1.year
        ) last_race ON ds.raceId = (
            SELECT r2.raceId
            FROM races r2
            WHERE r2.year = last_race.year AND r2.date = last_race.last_race_date
            LIMIT 1
        )
        INNER JOIN races r ON ds.raceId = r.raceId AND r.date = last_race.last_race_date
        INNER JOIN drivers d ON ds.driverId = d.driverId;
    '''

    points_df = pd.read_sql_query(points_query, conn)

    points_df = points_df.sort_values(by=['year', 'points'], ascending=[True, False])

    points_df['place'] = points_df.groupby('year')['points'].rank(ascending=False, method='min').astype(int)

    return points_df

# Example usage
result_df = calculate_driver_places()
print(result_df)
